In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../../Data/cleanedNotRecoded.csv")

df = data.copy()

df = df.drop(["ID", "Customer_ID", "Name", "SSN", "Occupation"], axis=1)

df["Credit_Score"] = df["Credit_Score"].map({"Good":2, "Standard":1, "Poor":0})
df["Credit_Mix"] = df["Credit_Mix"].map({"Good":2, "Standard":1, "Bad":0})

df = pd.get_dummies(df, columns=['Payment_of_Min_Amount', 'Payment_Behaviour'], drop_first=True)

In [3]:
from sklearn.ensemble import IsolationForest

iforestModel = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

iforestModel.fit(df)

# Predict anomalies (-1 for outliers, 1 for inliers)
df['anomaly'] = iforestModel.predict(df)

df = df[df['anomaly'] == 1].drop(columns=['anomaly'])

In [4]:
X = df.drop(["Credit_Score"], axis=1)
y = df["Credit_Score"]

In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [6]:
X.T

,0,1,2,3,4,5,6,7,8,9,...,130253,130254,130255,130256,130257,130258,130259,130260,130261,130262
Month,1,2,3,4,5,6,7,8,1,2,...,4,7,2,3,7,3,3,8,3,6
Age,23,23,23,23,23,23,23,23,28,28,...,38,44,36,39,46,25,49,32,48,52
Annual_Income,19114.12,19114.12,19114.12,19114.12,19114.12,19114.12,19114.12,19114.12,34847.84,34847.84,...,86767.810708,87655.112045,30768.659465,75557.992086,17132.905,28835.68,44066.78,27681.402301,116396.67,30365.13
Monthly_Inhand_Salary,1824.84,1824.84,1824.84,1824.84,1824.84,1824.84,1824.84,1824.84,3037.99,3037.99,...,7120.74623,7390.159332,2666.233153,6072.692398,1283.74,2373.97,3424.23,2398.722349,9501.72,2353.43
Num_Bank_Accounts,3,3,3,3,3,3,3,3,2,2,...,3,2,4,5,1,2,5,7,0,4
Num_Credit_Card,4,4,4,4,4,4,4,4,4,4,...,6,5,5,3,5,5,4,4,4,1
Interest_Rate,3,3,3,3,3,3,3,3,6,6,...,7,7,6,7,5,8,6,10,7,7
Num_of_Loan,4,4,4,4,4,4,4,4,1,1,...,1,4,3,3,2,4,4,2,3,3
Type_of_Loan,0.076445,0.076451,0.076448,0.076449,0.076449,0.076449,0.076449,0.076449,-0.084963,-0.084963,...,-0.093388,-0.029817,-0.153154,-0.147979,-0.109381,0.289985,-0.417693,0.302257,-0.0653,-0.262553
Delay_from_due_date,3,-1,3,5,6,8,3,3,3,7,...,12,5,9,23,5,13,6,28,3,14


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
import optuna
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [10]:
from sklearn.linear_model import LogisticRegression


def objective(trial):
    
    C = trial.suggest_float('C', 1e-6, 1e2, log = True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'newton-cg', 'lbfgs'])
    
    if solver in ['liblinear']:
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    else:
        penalty = 'l2'
    
    LogModel = LogisticRegression(
        C=C,
        solver=solver,
        penalty=penalty,
        multi_class='auto',
        random_state=42,
        max_iter=1000
    )
    
    cv_scores = cross_val_score(LogModel, X_train, y_train, cv=stratified_kfold, n_jobs=-1, scoring='accuracy')
    return cv_scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print(f"Best hyperparameters: {study.best_params}")
print(f"Best cross-validation score: {study.best_value}")

[I 2024-08-24 22:29:30,279] A new study created in memory with name: no-name-4a5f96b6-b3bf-4f30-98a2-02f68dcb8fc1
[I 2024-08-24 22:29:36,566] Trial 0 finished with value: 0.7246041646674983 and parameters: {'C': 6.359102878107446, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.7246041646674983.
[I 2024-08-24 22:29:38,295] Trial 1 finished with value: 0.6232511275309471 and parameters: {'C': 3.650332676154276e-06, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.7246041646674983.
[I 2024-08-24 22:29:53,633] Trial 2 finished with value: 0.7216869782170617 and parameters: {'C': 0.00414171318751207, 'solver': 'liblinear', 'penalty': 'l1'}. Best is trial 0 with value: 0.7246041646674983.
[I 2024-08-24 22:29:57,272] Trial 3 finished with value: 0.7247481047884081 and parameters: {'C': 5.751693065071367, 'solver': 'lbfgs'}. Best is trial 3 with value: 0.7247481047884081.
[I 2024-08-24 22:30:45,150] Trial 4 finished with value: 0.725592553497745 and parameters: {'C': 0.011580624726

Best hyperparameters: {'C': 0.0022373165347048915, 'solver': 'newton-cg'}
Best cross-validation score: 0.7283657998272718


In [11]:
bestLogModel = LogisticRegression(**study.best_params, multi_class='auto')

bestLogModel.fit(X_train, y_train)



y_pred = bestLogModel.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {test_accuracy}")

Test set accuracy: 0.7252523701685026


In [12]:
y_pred

array([0, 1, 2, ..., 2, 1, 0], dtype=int64)

In [13]:
bestLogModel = LogisticRegression(**study.best_params, multi_class='auto', random_state=42, max_iter=1000)

bestLogModel.fit(X_train, y_train)

LogisticRegression(C=0.0022373165347048915, max_iter=1000, random_state=42,
                   solver='newton-cg')

In [14]:
y_pred = bestLogModel.predict(X_train)
test_accuracy = accuracy_score(y_train, y_pred)
print(f"Test set accuracy: {test_accuracy}")

Test set accuracy: 0.7284041838595144


In [29]:
import xgboost as xgb

def objective(trial):
    param = {
        'objective': 'multi:softprob',
        'num_class': 3,
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 0.1, log = True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'n_jobs': -1
    }

    XGBmodel = xgb.XGBClassifier(**param, use_label_encoder=False)
    
    score = cross_val_score(XGBmodel, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    return score.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print(f"Best hyperparameters: {study.best_params}")
print(f"Best cross-validation score: {study.best_value}")

[I 2024-08-25 02:12:06,839] A new study created in memory with name: no-name-89b41e2b-56a6-4e11-9731-6e9d77dbe6f9
[I 2024-08-25 02:12:49,400] Trial 0 finished with value: 0.7574033202187889 and parameters: {'lambda': 0.000182857327979409, 'alpha': 0.9283605562727094, 'subsample': 0.8056571734951055, 'colsample_bytree': 0.858772347407623, 'learning_rate': 1.868588829024561e-07, 'max_depth': 7, 'n_estimators': 723, 'min_child_weight': 8}. Best is trial 0 with value: 0.7574033202187889.
[I 2024-08-25 02:13:12,121] Trial 1 finished with value: 0.7567124076384224 and parameters: {'lambda': 1.1198207733270591e-05, 'alpha': 0.001831900270569076, 'subsample': 0.6777603442732103, 'colsample_bytree': 0.6199114342790495, 'learning_rate': 0.0016131280805200994, 'max_depth': 7, 'n_estimators': 344, 'min_child_weight': 7}. Best is trial 0 with value: 0.7574033202187889.
[I 2024-08-25 02:13:40,975] Trial 2 finished with value: 0.7783322137990595 and parameters: {'lambda': 0.04226505114385148, 'alpha'

Best hyperparameters: {'lambda': 2.806695026145806e-08, 'alpha': 1.9793765374390634e-05, 'subsample': 0.8868657639446468, 'colsample_bytree': 0.6370536771529123, 'learning_rate': 0.09788954176891546, 'max_depth': 9, 'n_estimators': 597, 'min_child_weight': 1}
Best cross-validation score: 0.8941944151233088


In [30]:
bestXGBmodel = xgb.XGBClassifier(**study.best_params, use_label_encoder=False)
bestXGBmodel.fit(X_train, y_train)

y_pred = bestXGBmodel.predict(X_train)

print("=====Train======")
print("Accuracy:", accuracy_score(y_train, y_pred))
print("f1_score:", f1_score(y_train, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_pred))


y_pred = bestXGBmodel.predict(X_test)

print("=====Test======")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred))

=====Train======
Accuracy: 0.9985893868150849
f1_score: 0.9985888280168681
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     34649
           1       1.00      1.00      1.00     34754
           2       1.00      1.00      1.00     34807

    accuracy                           1.00    104210
   macro avg       1.00      1.00      1.00    104210
weighted avg       1.00      1.00      1.00    104210

=====Test======
Accuracy: 0.9020074463593444
f1_score: 0.9011113611039738
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.92      0.90      8772
           1       0.89      0.82      0.85      8667
           2       0.93      0.96      0.95      8614

    accuracy                           0.90     26053
   macro avg       0.90      0.90      0.90     26053
weighted avg       0.90      0.90      0.90     26053



In [25]:
from sklearn.neighbors import KNeighborsClassifier

def objective(trial):

    n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
    p = trial.suggest_categorical('p', [1, 2])  # 1 for Manhattan distance, 2 for Euclidean distance
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])

    knnModel = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        p=p,
        weights=weights
    )

    cv_scores = cross_val_score(knnModel, X_train, y_train, cv=stratified_kfold, n_jobs=-1, scoring='accuracy')
    return cv_scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print(f"Best hyperparameters: {study.best_params}")
print(f"Best cross-validation score: {study.best_value}")

[I 2024-08-25 01:16:11,791] A new study created in memory with name: no-name-e2e709f5-575e-4251-8c28-5d706259f918
[I 2024-08-25 01:16:20,472] Trial 0 finished with value: 0.759687170137223 and parameters: {'n_neighbors': 18, 'p': 2, 'weights': 'uniform'}. Best is trial 0 with value: 0.759687170137223.
[I 2024-08-25 01:16:28,662] Trial 1 finished with value: 0.7652336627962768 and parameters: {'n_neighbors': 47, 'p': 2, 'weights': 'distance'}. Best is trial 1 with value: 0.7652336627962768.
[I 2024-08-25 01:16:36,792] Trial 2 finished with value: 0.7741003742443144 and parameters: {'n_neighbors': 28, 'p': 2, 'weights': 'distance'}. Best is trial 2 with value: 0.7741003742443144.
[I 2024-08-25 01:16:44,545] Trial 3 finished with value: 0.762402840418386 and parameters: {'n_neighbors': 14, 'p': 2, 'weights': 'uniform'}. Best is trial 2 with value: 0.7741003742443144.
[I 2024-08-25 01:17:06,994] Trial 4 finished with value: 0.7769216006141445 and parameters: {'n_neighbors': 23, 'p': 1, 'we

Best hyperparameters: {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Best cross-validation score: 0.8450628538527972


In [31]:
bestKNNModel = KNeighborsClassifier(**study.best_params)
bestKNNModel.fit(X_train, y_train)


y_pred = bestKNNModel.predict(X_train)

print("=====Train======")
print("Accuracy:", accuracy_score(y_train, y_pred))
print("f1_score:", f1_score(y_train, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_pred))


y_pred = bestKNNModel.predict(X_test)

print("=====Test======")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred))

=====Train======
Accuracy: 1.0
f1_score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     34649
           1       1.00      1.00      1.00     34754
           2       1.00      1.00      1.00     34807

    accuracy                           1.00    104210
   macro avg       1.00      1.00      1.00    104210
weighted avg       1.00      1.00      1.00    104210

=====Test======
Accuracy: 0.8567535408590181
f1_score: 0.854189241032125
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.87      8772
           1       0.86      0.73      0.79      8667
           2       0.86      0.96      0.91      8614

    accuracy                           0.86     26053
   macro avg       0.86      0.86      0.85     26053
weighted avg       0.86      0.86      0.85     26053



In [33]:
from sklearn.tree import DecisionTreeClassifier

def objective(trial):
   
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    dtModel = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features
    )
    
    cv_scores = cross_val_score(dtModel, X_train, y_train, cv=stratified_kfold, n_jobs=-1, scoring='accuracy')
    return cv_scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters:', study.best_params)
print('Best accuracy:', study.best_value)

[I 2024-08-25 03:56:57,001] A new study created in memory with name: no-name-0963d467-5cf2-4826-8376-f56ded10bf76
[I 2024-08-25 03:56:59,863] Trial 0 finished with value: 0.7903368198829287 and parameters: {'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 0 with value: 0.7903368198829287.
[I 2024-08-25 03:57:01,273] Trial 1 finished with value: 0.6734094616639477 and parameters: {'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 14, 'max_features': 'log2'}. Best is trial 0 with value: 0.7903368198829287.
[I 2024-08-25 03:57:02,483] Trial 2 finished with value: 0.6406582861529604 and parameters: {'max_depth': 3, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 'log2'}. Best is trial 0 with value: 0.7903368198829287.
[I 2024-08-25 03:57:04,133] Trial 3 finished with value: 0.7743786584780731 and parameters: {'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 16, 'max_features': None}. Best is trial 0 

Best hyperparameters: {'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None}
Best accuracy: 0.7950100758084637


In [34]:
bestDtModel = DecisionTreeClassifier(**study.best_params)
bestDtModel.fit(X_train, y_train)


y_pred = bestDtModel.predict(X_train)

print("=====Train======")
print("Accuracy:", accuracy_score(y_train, y_pred))
print("f1_score:", f1_score(y_train, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_pred))


y_pred = bestDtModel.predict(X_test)

print("=====Test======")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred))

=====Train======
Accuracy: 0.8833029459744747
f1_score: 0.8822629316698022
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.91      0.89     34649
           1       0.88      0.80      0.84     34754
           2       0.89      0.94      0.91     34807

    accuracy                           0.88    104210
   macro avg       0.88      0.88      0.88    104210
weighted avg       0.88      0.88      0.88    104210

=====Test======
Accuracy: 0.8020957279392008
f1_score: 0.8001514546597022
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.84      0.82      8772
           1       0.77      0.69      0.73      8667
           2       0.83      0.87      0.85      8614

    accuracy                           0.80     26053
   macro avg       0.80      0.80      0.80     26053
weighted avg       0.80      0.80      0.80     26053



In [9]:
from sklearn.ensemble import ExtraTreesClassifier

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    
    etModel = ExtraTreesClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    cv_scores = cross_val_score(etModel, X_train, y_train, cv=stratified_kfold, n_jobs=-1, scoring='accuracy')
    return cv_scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters:', study.best_params)
print('Best accuracy:', study.best_value)

[I 2024-08-26 03:41:16,296] A new study created in memory with name: no-name-d0936fc7-8b29-41b5-b908-b497a116b27a
[I 2024-08-26 03:41:29,629] Trial 0 finished with value: 0.7792150465406391 and parameters: {'n_estimators': 129, 'max_depth': 20, 'min_samples_split': 16, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7792150465406391.
[I 2024-08-26 03:41:47,242] Trial 1 finished with value: 0.7972267536704731 and parameters: {'n_estimators': 168, 'max_depth': 46, 'min_samples_split': 13, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.7972267536704731.
[I 2024-08-26 03:42:01,168] Trial 2 finished with value: 0.7333749160349294 and parameters: {'n_estimators': 121, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.7972267536704731.
[I 2024-08-26 03:42:17,357] Trial 3 finished with value: 0.743441128490548 and parameters: {'n_estimators': 89, 'max_depth': 12, 'min_samples_split': 12, 'min_samples_leaf': 12}. Best is trial 1 with valu

Best hyperparameters: {'n_estimators': 108, 'max_depth': 42, 'min_samples_split': 3, 'min_samples_leaf': 1}
Best accuracy: 0.86750791670665


In [11]:
bestEtModel = ExtraTreesClassifier(**study.best_params)
bestEtModel.fit(X_train, y_train)


y_pred = bestEtModel.predict(X_train)

print("=====Train======")
print("Accuracy:", accuracy_score(y_train, y_pred))
print("f1_score:", f1_score(y_train, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_pred))


y_pred = bestEtModel.predict(X_test)

print("=====Test======")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred))

=====Train======
Accuracy: 0.959332117838979
f1_score: 0.9592738071951241
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97     34649
           1       0.97      0.93      0.95     34754
           2       0.94      0.98      0.96     34807

    accuracy                           0.96    104210
   macro avg       0.96      0.96      0.96    104210
weighted avg       0.96      0.96      0.96    104210

=====Test======
Accuracy: 0.8401719571642421
f1_score: 0.8384771796456604
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.85      0.86      8772
           1       0.84      0.74      0.79      8667
           2       0.83      0.93      0.87      8614

    accuracy                           0.84     26053
   macro avg       0.84      0.84      0.84     26053
weighted avg       0.84      0.84      0.84     26053



In [ ]:
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    
    
    rfModel = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    cv_scores = cross_val_score(rfModel, X_train, y_train, cv=stratified_kfold, n_jobs=-1, scoring='accuracy')
    return cv_scores.mean()
    
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters:', study.best_params)
print('Best accuracy:', study.best_value)

In [9]:
from sklearn.ensemble import RandomForestClassifier

rfModel = RandomForestClassifier(n_estimators=200)

rfModel.fit(X_train, y_train)


y_pred = rfModel.predict(X_train)

print("=====Train======")
print("Accuracy:", accuracy_score(y_train, y_pred))
print("f1_score:", f1_score(y_train, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_pred))

scores = cross_val_score(rfModel, X_train, y_train, cv=stratified_kfold, scoring='accuracy') 

print(f"Cross-validation accuracy scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")

y_pred = rfModel.predict(X_test)

print("=====Test======")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred))

=====Train======
Accuracy: 1.0
f1_score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     34649
           1       1.00      1.00      1.00     34754
           2       1.00      1.00      1.00     34807

    accuracy                           1.00    104210
   macro avg       1.00      1.00      1.00    104210
weighted avg       1.00      1.00      1.00    104210

Cross-validation accuracy scores: [0.87462815 0.87510796 0.87208521 0.87386047 0.87371653]
Mean accuracy: 0.8738796660589196
=====Test======
Accuracy: 0.8806663340114382
f1_score: 0.8790769007700834
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.91      0.89      8772
           1       0.88      0.78      0.83      8667
           2       0.89      0.95      0.92      8614

    accuracy                           0.88     26053
   macro avg       0.88      0.88      0.88     26053
w